In [1]:
import pyodbc
import glob

# Define a function to create the new table
def create_new_table(cursor, source_table_name):
    text_cols = ['DYCOL03', 'TakmilDescC']
    # Get the column names from the source table
    cursor.execute(f'SELECT * FROM {source_table_name}')
    column_names = [column[0] for column in cursor.description]
    # exclude text columns
    if source_table_name.endswith('P3S13') or source_table_name.endswith('Data'):
        excluded_columns = [item for item in column_names if item in text_cols]
        column_names = [item for item in column_names if item not in text_cols]
        select_clause = ', '.join([f'IIF(ISNUMERIC([{source_table_name}].[{column_name}]), [{source_table_name}].[{column_name}] * 1, 0) AS {column_name}' for column_name in column_names])
        for column_name in excluded_columns:
            # add text columns again
            select_clause += f', [{source_table_name}].[{column_name}] AS {column_name}'
    else:
        select_clause = ', '.join([f'IIF(ISNUMERIC([{source_table_name}].[{column_name}]), [{source_table_name}].[{column_name}] * 1, 0) AS {column_name}' for column_name in column_names])
    into_clause = f'N{source_table_name}'
    sql_command = f'SELECT {select_clause} INTO {into_clause} FROM {source_table_name}'

    # Create a new table with all columns from the source table
    cursor.execute(sql_command)

# Define a function to delete the old table
def delete_old_table(cursor, source_table_name):
    cursor.execute(f'DROP TABLE {source_table_name}')

# Define main function
def main(file_path):
    file_path = str(file_path)
    # Set up the connection string
    conn_str = str(
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        rf'DBQ={file_path};'
    )

        # Connect to the MS Access database
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        table_names = [x.table_name for x in cursor.tables() if x.table_type == 'TABLE']
        for table in table_names:
            # Call the function to create the new table
            create_new_table(cursor, table)

            # Call the function to replace null values with zero
            #replace_null_values(cursor, '1400test')

            # Call the function to delete the old table
            delete_old_table(cursor, table)


In [ ]:
# specify the directory to search in
directory = r"C:\Users\user\Downloads\a_folder"        # put mdb files folder address
# find all .mdb files in the directory
mdb_files = glob.glob(directory + '/*.mdb')

for file in mdb_files:
    main(file)